In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver

In [14]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://www.mobileindex.com/mi-chart/top-100/top-games')
time.sleep(3)
game_list = []
num = 0
tran_before =['Role Playing', 'Sports', 'Action', 'Strategy', 'Puzzle', 'Simulation', 'Music', 'Adventure', 'Casino', 'Arcade', 'Board', 'Card']
tran_after = ['롤플레잉', '스포츠', '액션', '전략', '퍼즐', '시뮬레이션', '음악', '어드벤처', '카지노', '아케이드', '보드', '카드']
while True:
    num+=1
# 게임정보를 추출하고 원래화면으로 돌아오기위해 url 재설정
    driver.get('https://www.mobileindex.com/mi-chart/top-100/top-games')
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
# 게임명 추출작업
    a_list = soup.select(f'#page-scroll-obj > section > table > tbody > tr:nth-child({num}) > td:nth-child(2) > span > span > span:nth-child(1)')
# 마지막 게임정보를 추출하고 더이상 정보가 없을경우 while문 탈출
    if not a_list:
        break
    title = a_list[0].text.replace(',','/') # 게임제목에 쉼표가 들어갈 경우 csv로 만들어서 SQL에 넣을때 오류가 발생하기에 쉼표를 /로 교체
# 게임제목외 데이터를 추출하기 위해 클릭이 필요, 중간에 클릭이 안되는 것이 존재할경우 그 항목을 무시하고 진행    
    try :
        driver.find_element_by_css_selector(f'#page-scroll-obj > section > table > tbody > tr:nth-child({num}) > td:nth-child(2)').click()
    except:
        continue
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
# 게임장르가 구글스토어, 앱스토어 각각 한개씩 나와있으며 같은경우도, 다른경우도 존재하기에 정제작업 및 구글스토어나, 앱스토어 한곳에만
# 존재하는 경우 오류가 발생하여 except문으로 빈값을 줌
    try : 
        b_list = soup.select('#page-floating-scroll-page > section > div.detail > table > tbody > tr:nth-child(1) > td:nth-child(2)')
        genre1 = b_list[0].text
    except : genre1 = '-'
    try :
        c_list = soup.select('#page-floating-scroll-page > section > div.detail > table > tbody > tr:nth-child(2) > td:nth-child(2)')
        genre2 = c_list[0].text
        if genre2 in tran_before:
            genre2 = tran_after[tran_before.index(genre2)]
    except : genre2 = '-'
    if genre1 == genre2 :
        genre2 = '-'
    game_list.append([num, title, genre1, genre2])
game_list


[[1, '오딘: 발할라 라이징', '롤플레잉', '액션'],
 [2, '리니지M', '롤플레잉', '-'],
 [3, '블레이드&소울2', '롤플레잉', '-'],
 [4, '삼국지 전략판', '전략', '시뮬레이션'],
 [5, '리니지2M', '롤플레잉', '액션'],
 [6, '원신-1주년', '어드벤처', '롤플레잉'],
 [7, 'FIFA ONLINE 4 M by EA SPORTS™', '스포츠', '시뮬레이션'],
 [8, '기적의 검', '롤플레잉', '액션'],
 [9, '히어로즈\xa0테일즈', '롤플레잉', '어드벤처'],
 [10, '제2의 나라: CROSS WORLDS', '롤플레잉', '어드벤처'],
 [11, '쿠키런: 킹덤', '롤플레잉', '-'],
 [12, '바람의나라: 연', '롤플레잉', '액션'],
 [13, '라이즈 오브 킹덤즈', '전략', '롤플레잉'],
 [14, '블레이드&소울 레볼루션', '롤플레잉', '-'],
 [15, 'MARVEL 퓨처 레볼루션', '롤플레잉', '-'],
 [16, '세븐나이츠2', '롤플레잉', '-'],
 [17, '뮤 아크엔젤', '롤플레잉', '-'],
 [18, '카트라이더 러쉬플러스', '자동차 경주', '스포츠'],
 [19, 'ROBLOX', '어드벤처', '-'],
 [20, 'S.O.S: 스테이트 오브 서바이벌 x 더 워킹 데드', '전략', '-'],
 [21, '한게임 포커', '카지노', '카드'],
 [22, '나의 히어로 아카데미아: 최강 히어로', '액션', '-'],
 [23, '에픽세븐', '롤플레잉', '전략'],
 [24, '피파모바일', '스포츠', '-'],
 [25, 'V4', '롤플레잉', '-'],
 [26, 'R2M', '롤플레잉', '-'],
 [27, '리니지2 레볼루션', '-', '-'],
 [28, '천상나르샤', '롤플레잉', '-'],
 [29, '뮤 아크엔젤2', '롤플레잉', '-'],
 [30, '월드 플리퍼', '액션',

In [17]:
frame = ['랭킹', '게임이름', '장르1', '장르2']
df = pd.DataFrame(game_list, columns=frame)
df

,랭킹,게임이름,장르1,장르2
0,1,오딘: 발할라 라이징,롤플레잉,액션
1,2,리니지M,롤플레잉,-
2,3,블레이드&소울2,롤플레잉,-
3,4,삼국지 전략판,전략,시뮬레이션
4,5,리니지2M,롤플레잉,액션
...,...,...,...,...
195,196,행복의 저택,퍼즐,-
196,197,애니팡 포커,카지노,카드
197,198,닌자대전,롤플레잉,-
198,199,캔디크러쉬소다,캐주얼 게임,아케이드


In [ ]:
df.to_csv('project/mobile_kor_data.csv', index=False)